In [1]:
import pyspark
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pymongo import MongoClient

```bash
spark-submit --packages org.mongodb.spark:mongo-spark-connector_2.12:10.5.0,org.postgresql:postgresql:42.7.1 notebooks/spark.py
```
run this in terminal to correctly execute the process and make sure that the version 10.5.0 of the connector is installed

In [ ]:
# conf = pyspark.SparkConf().setMaster("local").setAppName("test").setAll([('spark.driver.memory', '40g'),('spark.executor.memory','50g')])
# sc = SparkContext(conf=conf)
# sqlC = SQLContext(sc)

In [ ]:
# train_df = sqlC.read.format('com.mongodb.spark.sql.DefaultSource').option("uri", "mongodb://localhost:27017.tickets").load()

In [2]:
client = MongoClient("mongodb://localhost:27017")
print(client.list_database_names())

['admin', 'config', 'local', 'raw']


In [3]:


# Create a Spark session
spark = SparkSession.builder \
    .appName("MongoSparkStreamingIntegration") \
    .config("spark.jars.packages", "org.mongodb.spark:mongo-spark-connector_2.12:10.1.1") \
    .config("spark.mongodb.input.uri", "mongodb://localhost:27017/raw.passengers") \
    .getOrCreate()
spark.sparkContext.setLogLevel("INFO")
# Read from MongoDB as a streaming DataFrame
mongo_stream_df = spark.readStream \
    .format("mongodb") \
    .option("uri", "mongodb://localhost:27017/raw.passengers") \
    .load()

# Show the streaming DataFrame (just for debug purposes)
mongo_stream_df.select("bus_id", "predicted_passengers_in").writeStream \
    .outputMode("append") \
    .format("console") \
    .start()

# Keep the stream active
spark.streams.awaitAnyTermination()


:: loading settings :: url = jar:file:/Users/anthony/sources/spark-3.5.5/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/anthony/.ivy2/cache
The jars for the packages stored in: /Users/anthony/.ivy2/jars
org.mongodb.spark#mongo-spark-connector_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-a55e84c2-ce4c-4f8b-9033-2bdeca4c7011;1.0
	confs: [default]
	found org.mongodb.spark#mongo-spark-connector_2.12;10.1.1 in central
	found org.mongodb#mongodb-driver-sync;4.8.2 in central
	[4.8.2] org.mongodb#mongodb-driver-sync;[4.8.1,4.8.99)
	found org.mongodb#bson;4.8.2 in central
	found org.mongodb#mongodb-driver-core;4.8.2 in central
	found org.mongodb#bson-record-codec;4.8.2 in central
:: resolution report :: resolve 412ms :: artifacts dl 3ms
	:: modules in use:
	org.mongodb#bson;4.8.2 from central in [default]
	org.mongodb#bson-record-codec;4.8.2 from central in [default]
	org.mongodb#mongodb-driver-core;4.8.2 from central in [default]
	org.mongodb#mongodb-driver-sync;4.8.2 from central in [default]
	org.mongodb.spark#mongo-spark-connect

Py4JJavaError: An error occurred while calling o34.load.
: com.mongodb.spark.sql.connector.exceptions.ConfigException: Missing configuration for: database
	at com.mongodb.spark.sql.connector.assertions.Assertions.validateConfig(Assertions.java:69)
	at com.mongodb.spark.sql.connector.config.AbstractMongoConfig.getDatabaseName(AbstractMongoConfig.java:111)
	at com.mongodb.spark.sql.connector.config.ReadConfig.getDatabaseName(ReadConfig.java:45)
	at com.mongodb.spark.sql.connector.config.AbstractMongoConfig.withCollection(AbstractMongoConfig.java:175)
	at com.mongodb.spark.sql.connector.config.ReadConfig.withCollection(ReadConfig.java:45)
	at com.mongodb.spark.sql.connector.schema.InferSchema.inferSchema(InferSchema.java:82)
	at com.mongodb.spark.sql.connector.MongoTableProvider.inferSchema(MongoTableProvider.java:62)
	at org.apache.spark.sql.execution.datasources.v2.DataSourceV2Utils$.getTableFromProvider(DataSourceV2Utils.scala:90)
	at org.apache.spark.sql.streaming.DataStreamReader.loa

In [ ]:
# # Start Spark with MongoDB connector
# spark = SparkSession.builder \
#     .appName("MongoSparkIntegration") \
#     .config("spark.jars.packages", "org.mongodb.spark:mongo-spark-connector_2.12:10.1.1") \
#     .config("spark.mongodb.read.connection.uri", "mongodb://localhost:27017") \
#     .config("spark.mongodb.read.database", "raw") \
#     .config("spark.mongodb.read.collection", "passengers") \
#     .getOrCreate()

In [ ]:
print(spark.sparkContext._jsc.sc().listJars())

In [ ]:
# Read from MongoDB
mongo_df = spark.read.format("mongodb").load()

In [ ]:
mongo_df.select("bus_id", "predicted_passengers_in").show()

In [ ]:
mongo_df.show(1)

In [ ]:
spark.stop()

In [ ]:
# For v3.x and later
mongo_df = spark.read.format("mongodb").option("collection", "tickets").load()

In [ ]:
mongo_df = spark.read.format("mongodb").option("spark.mongodb.read.database", "raw").option("spark.mongodb.read.collection", "tickets").load()

In [ ]:
print(spark.version)

In [ ]:
mongo_df = spark.read.format("mongodb").load()